## Задание по программированию: Рекомендательные системы

### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.  

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

### Важно:

    Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
    Если товар не встречался в обучающей выборке, его популярность равна 0.
    Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
    Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.
    
### Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

2. Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие?           Обратите внимание на различие качества на обучающей и тестовой           выборке в случае рекомендаций по частотам покупки.



Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Дополнительные вопросы

    Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
    Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.



In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('coursera_sessions_train.txt', 'r') as f:
    train = f.read().splitlines()
with open('coursera_sessions_test.txt', 'r') as f:
    test = f.read().splitlines()

In [3]:
train[0:5]

['0,1,2,3,4,5;',
 '9,10,11,9,11,12,9,11;',
 '16,17,18,19,20,21;',
 '24,25,26,27,24;',
 '34,35,36,34,37,35,36,37,38,39,38,39;']

In [4]:
test[0:5]

['6,7,8;', '13,14,15;', '22,23;', '28,29,30,31,32,33;', '40,41;']

In [5]:
# l - look, p - purchase
def look_purchase(file):
    file_lp = []
    for session in file:
        l_items, p_items = session.split(';')
        l_items = map(int, l_items.split(','))
        if len(p_items) > 0:
            p_items = map(int, p_items.split(','))
        else:
            p_items = []
        file_lp.append([l_items, p_items])
        return file_lp

In [6]:
train_lp = look_purchase(train)
test_lp = look_purchase(test)

In [12]:
# Array of looks
train_l = [row[0] for row in train_lp]
train_l_np = np.array( [id_n for session in train_l for id_n in session] )

# Array of unique ids and looks in train data
train_l_cnt = np.transpose(np.unique(train_l_np, return_counts=True))

In [13]:
train_l_cnt

array([[0, 1],
       [1, 1],
       [2, 1],
       [3, 1],
       [4, 1],
       [5, 1]])

In [14]:
# Array of purchases
train_p = [row[1] for row in train_lp]
train_p_np = np.array( [id_n for session in train_p for id_n in session] )

# Array of unique ids and purchases in train dataset
train_p_cnt = np.transpose(np.unique(train_p_np, return_counts=True))

In [15]:
train_p_cnt

array([], shape=(0, 2), dtype=float64)

In [17]:
# Sorting arrays of looks and purchases by counts
train_l_cnt = train_l_cnt[train_l_cnt[:,1].argsort()][::-1]
train_p_cnt = train_p_cnt[train_p_cnt[:,1].argsort()][::-1]

In [18]:
def prec_rec_metrics(session, reccomendations, k):
    purchase = 0
    for ind in reccomendations:
        if ind in session:
            purchase += 1 
    precision = purchase / k
    recall = purchase / len(session)
    return(precision, recall)

In [19]:
# Calculate metrics for train dataset, suggestions based on looks
prec_at_1_tr_l, rec_at_1_tr_l = [], []
prec_at_5_tr_l, rec_at_5_tr_l = [], []
k1, k5 = 1, 5
for i, sess_p in enumerate(train_p):
    # skip sessions without purchases
    if sess_p == []: continue
    
    # looks ids
    sess_l = train_l[i]

    # sorted looks ids indices in sess_train_l_cnt array
    # sort in accordance with looks counts
    l_ind_sess = []
    for j in range(len(sess_l)):
        l_ind_sess.append(np.where(train_l_cnt[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    # k1 recommendations
    num_of_recs_k1 = min(k1, len(sess_l))
    if num_of_recs_k1 == 0: continue
    recs_k1 = train_l_cnt[l_ind_sess_sorted[:num_of_recs_k1],0]
    
    # k1 metrics
    prec_1, rec_1 = prec_rec_metrics(sess_p, recs_k1, k1)
    prec_at_1_tr_l.append(prec_1)
    rec_at_1_tr_l.append(rec_1)
    
    # k5 recommendations
    num_of_recs_k5 = min(k5, len(sess_l))
    if num_of_recs_k5 == 0: continue
    recs_k5 = train_l_cnt[l_ind_sess_sorted[:num_of_recs_k5],0]
    
    # k5 metrics
    prec_5, rec_5 = prec_rec_metrics(sess_p, recs_k5, k5)
    prec_at_5_tr_l.append(prec_5)
    rec_at_5_tr_l.append(rec_5)

In [21]:
avg_prec_at_1_tr_l = np.mean(prec_at_1_tr_l)
avg_rec_at_1_tr_l = np.mean(rec_at_1_tr_l)
avg_prec_at_5_tr_l = np.mean(prec_at_5_tr_l)
avg_rec_at_5_tr_l = np.mean(rec_at_5_tr_l)

In [22]:
with open('ans1.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_tr_l, 2)
    p1 = '%.2f' % round(avg_prec_at_1_tr_l, 2)
    r5 = '%.2f' % round(avg_rec_at_5_tr_l, 2)
    p5 = '%.2f' % round(avg_prec_at_5_tr_l, 2)
    ans1 = ' '.join([r1, p1, r5, p5])
    print('Answer 1:', ans1)
    f.write(ans1)

Answer 1: nan nan nan nan


In [25]:
with open('ans4.txt', 'w') as f:
    r1 = '%.2f' % round(avg_rec_at_1_tr_l, 2)
    p1 = '%.2f' % round(avg_prec_at_1_tr_l, 2)
    r5 = '%.2f' % round(avg_rec_at_5_tr_l, 2)
    p5 = '%.2f' % round(avg_prec_at_5_tr_l, 2)
    ans3 = ' '.join([r1, p1, r5, p5])
    print('Answer 1:', ans2)
    f.write(ans3)

Answer 1: nan nan nan nan
